<a href="https://colab.research.google.com/github/gyulgyulyan/MyScripts/blob/main/LTV_150_ad_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import os
import datetime as dt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from scipy.misc import derivative
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import re
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
import lightgbm as lgb

from sqlalchemy import create_engine # !pip install clickhouse-sqlalchemy

In [ ]:
# выгрузка данных
df_merged=pd.read_csv('ml_em_jan_new.csv')

In [ ]:
df0=pd.read_csv('ml_sm_jan_new0.csv', low_memory=False)
df1=pd.read_csv('ml_sm_jan_new1.csv', low_memory=False)
df2=pd.read_csv('ml_sm_jan_new2.csv', low_memory=False)
df3=pd.read_csv('ml_sm_jan_new3.csv', low_memory=False)
df4=pd.read_csv('ml_sm_jan_new4.csv', low_memory=False)
df5=pd.read_csv('ml_sm_jan_new5.csv', low_memory=False)
df6=pd.read_csv('ml_sm_jan_new6.csv', low_memory=False)

In [ ]:
df_merged=df_merged.append(df0)
df_merged=df_merged.append(df1)
df_merged=df_merged.append(df2)
df_merged=df_merged.append(df3)
df_merged=df_merged.append(df4)
df_merged=df_merged.append(df5)
df_merged=df_merged.append(df6)

In [ ]:
df_merged_ih=pd.read_csv('ml_ih_jan_new.csv')

In [ ]:
df_merged=df_merged.append(df_merged_ih)

In [ ]:
quint=pd.read_csv('train_quantile.csv')
df_merged=pd.merge(df_merged, quint, on=["app_id", "country_installs"], how='left')

In [ ]:
# выгрузка данных
df_merged['no_activity_last_3_days'] = df_merged.apply(lambda x: (1 if x['num_sessions_11'] * x['num_sessions_12'] * x['num_sessions_13'] > 0 else 0), axis = 1)
df_merged['install_date_utc'] = pd.to_datetime(df_merged['install_date_utc'])
df_merged['last_session_0_13'] = pd.to_datetime(df_merged['last_session_0_13'], format='%Y-%m-%d')
df_merged['max_date'] = df_merged['install_date_utc'] + dt.timedelta(14) 
df_merged['days_from_last_session'] = df_merged.apply(lambda x: x['max_date']-x['last_session_0_13'], axis=1)
df_merged['days_from_last_session'] = df_merged['days_from_last_session'].dt.days

In [ ]:
df_merged['days_from_last_session'] = df_merged.apply(lambda x: 0 if x['days_from_last_session'] <0 else x['days_from_last_session'], axis = 1)
df_merged['days_from_last_session'] = df_merged.apply(lambda x: 13 if x['days_from_last_session']>13 else x['days_from_last_session'], axis = 1)

In [ ]:
df_merged['daylife'] = df_merged.apply(lambda x: x['last_session_0_13']-x['install_date_utc'], axis=1)
df_merged['daylife'] = df_merged['daylife'].dt.days
df_merged['daylife']=np.where((df_merged.max_das<14) & ((df_merged.daylife<0) | (df_merged.daylife>13)),df_merged.max_das, 13-df_merged.days_from_last_session)

In [ ]:
df_merged['app_revenue_0_13']=df_merged['app_revenue_0']+df_merged['app_revenue_1']+ df_merged['app_revenue_2']+ df_merged['app_revenue_3']+ df_merged['app_revenue_4']+df_merged['app_revenue_5']+ df_merged['app_revenue_6']+ df_merged['app_revenue_7']+ df_merged['app_revenue_8']+df_merged['app_revenue_9']+ df_merged['app_revenue_10']+ df_merged['app_revenue_11']+ df_merged['app_revenue_12']+df_merged['app_revenue_13']

In [ ]:
df_merged.loc[pd.isna(df_merged.campaign_id)==True,'campaign_id']=''

In [ ]:
features = ['uid_count', 'country_count', 'city_count', 'app_version_count', 
            'fs_visit_count', 'fs_visit_count_7_13', 'fs_visit_count_10_13', 
            'fs_visit_count_13', 'fs_visit_count_12', 'fs_visit_count_11',
            'is_signup', 'is_start_trial', 'ad_revenue_0','ad_revenue_1','ad_revenue_0_1',
       'ad_revenue_2', 'ad_revenue_3', 'ad_revenue_4', 'ad_revenue_5',
       'ad_revenue_6', 'ad_revenue_7', 'ad_revenue_8', 'ad_revenue_9',
       'ad_revenue_10', 'ad_revenue_11', 'ad_revenue_12', 'ad_revenue_13', 'ad_revenue_0_13', 
            'country_qol_idx', 'country_col_idx', 'is_capital',
            'app_revenue_0','app_revenue_0_1',
       'app_revenue_1', 'app_revenue_2', 'app_revenue_3', 'app_revenue_4',
       'app_revenue_5', 'app_revenue_6', 'app_revenue_7', 'app_revenue_8',
       'app_revenue_9', 'app_revenue_10', 'app_revenue_11', 'app_revenue_12',
       'app_revenue_13',
            'app_revenue_0_13',
            'das_count_0', 'das_count_1_3', 'das_count_4_7', 'das_count_7_13', 'das_count_0_13',
            'retention_0', 'retention_1', 'retention_2', 'retention_3', 'retention_4', 'retention_5',
            'retention_6', 'retention_7', 'retention_8', 'retention_9', 'retention_10', 'retention_11',
            'retention_12', 'retention_13',
            'num_sessions_0','num_sessions_1', 'num_sessions_2', 'num_sessions_3', 'num_sessions_4', 'num_sessions_5',
        'num_sessions_6', 'num_sessions_7', 'num_sessions_8', 'num_sessions_9', 'num_sessions_10', 'num_sessions_11',
        'num_sessions_12', 'num_sessions_13',
            'num_ads_0', 'num_ads_1', 'num_ads_2',
       'num_ads_3', 'num_ads_4', 'num_ads_5', 'num_ads_6', 'num_ads_7',
       'num_ads_8', 'num_ads_9', 'num_ads_10', 'num_ads_11', 'num_ads_12',
       'num_ads_13',
            
            'is_likely_bot', 'no_sessions', 
            'ads_per_session', 'income_per_ad',
            'num_sessions_bins', 'num_ads_bins',
            'num_dialogs_bins', 'photo_views_bins', 'dialogs_per_session', 'views_per_session',
            'has_install', 'niggers_group_0_13','vvp', 'daylife', 'sum_session_duration_0_13',
            'r_score','f_score','m_score','das_score','dial_score'
            ]

cat_features = ['country_signup_cat', 'media_source_cat', 'campaign_type_cat', 'top_age_group_', 
                'country_tier_cat', 'top_gender_cat', 'country_profile_cat', 'rfm']

#### Генерация всяких фичей

In [ ]:
df_merged['quan_20_das_count_0_13']=np.where(df_merged.quan_20_das_count_0_13>1,df_merged.quan_20_das_count_0_13,1)
df_merged['quan_20_num_dial_0_13']=np.where(df_merged.quan_20_num_dial_0_13>1,df_merged.quan_20_num_dial_0_13,1)

df_merged['r_score']=np.where(df_merged.days_from_last_session<=df_merged.quan_20_last_sess_0_13, 3,
                             np.where(df_merged.days_from_last_session<=df_merged.quan_80_last_sess_0_13, 2,1))
df_merged['f_score']=np.where(df_merged.num_sessions_0_13>df_merged.quan_80_sess_0_13, 3,
                             np.where(df_merged.num_sessions_0_13>1, 2,1))
df_merged['m_score']=np.where(df_merged.ad_revenue_0_13>df_merged.quan_80_ad_rev_0_13, 3,
                             np.where(df_merged.ad_revenue_0_13>0, 2,1))
df_merged['das_score']=np.where(df_merged.das_count_0_13>=df_merged.quan_80_das_count_0_13, 3,
                             np.where(df_merged.das_count_0_13>=df_merged.quan_20_das_count_0_13, 2,1))
df_merged['dial_score']=np.where(df_merged.days_from_last_session>=df_merged.quan_80_num_dial_0_13, 3,
                             np.where(df_merged.days_from_last_session>=df_merged.quan_20_num_dial_0_13, 2,1))

In [ ]:
df_merged['rfm'] = df_merged['r_score'].map(str) + df_merged['f_score'].map(str) + df_merged['m_score'].map(str) + df_merged['das_score'].map(str) + df_merged['dial_score'].map(str)
df_merged['rfm'] = df_merged['rfm'].astype('category')

In [ ]:
perc_sess = df_merged['num_sessions_0_13'].quantile([0.97]).values[0]
perc_ads = df_merged['num_ads_0_13'].quantile([0.97]).values[0]
df_merged['is_likely_bot'] = df_merged.apply(lambda x: (1 if (x['num_sessions_0_13'] >= perc_sess) or (x['num_ads_0_13'] >= perc_ads) else  0), axis = 1)
df_merged['no_sessions'] = df_merged.apply(lambda x: (1 if x['num_sessions_0_13'] == 0 else 0), axis = 1)

In [ ]:
df_merged['install_date_utc'] = pd.to_datetime(df_merged['install_date_utc'])
df_merged['ads_per_session'] = np.where(df_merged['num_sessions_0_13']==0,0,df_merged['num_ads_0_13'] / df_merged['num_sessions_0_13'])
df_merged['income_per_ad'] = np.where(df_merged['num_ads_0_13']==0,0,df_merged['ad_revenue_0_13'] / df_merged['num_ads_0_13'])
df_merged['dialogs_per_session'] = np.where(df_merged['num_sessions_0_13']==0,0,df_merged['num_dialogs_0_13'] / df_merged['num_sessions_0_13'])
df_merged['views_per_session'] = np.where(df_merged['num_sessions_0_13']==0,0,df_merged['photo_views_0_13'] / df_merged['num_sessions_0_13'])

def pct_rank_qcut(series, n):
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).argmax()
    return series.rank(pct=1).apply(f)

df_merged['num_sessions_bins'] = pct_rank_qcut(df_merged['num_sessions_0_13'], 10)
df_merged['num_ads_bins'] = pct_rank_qcut(df_merged['num_ads_0_13'], 10)
df_merged['num_dialogs_bins'] = pct_rank_qcut(df_merged['num_dialogs_0_13'], 10)
df_merged['photo_views_bins'] = pct_rank_qcut(df_merged['photo_views_0_13'], 10)

In [ ]:
vvp=pd.read_csv('vvp.csv', sep=";", low_memory=False)
df_merged=pd.merge(df_merged,vvp, how='left')
df_merged.loc[pd.isna(df_merged.vvp)==True,'vvp']=int(vvp.loc[vvp.country_profile=='unknown','vvp'])

In [ ]:
df_merged[features] = df_merged[features].fillna(0)

In [ ]:
df_em=df_merged.loc[(df_merged.app_id=='com.evermatch') & (df_merged.install_date_utc>="2022-01-24")].copy()
df_sm=df_merged.loc[(df_merged.app_id=='ru.fotostrana.sweetmeet') & (df_merged.install_date_utc>="2022-01-24")].copy()
df_ih=df_merged.loc[(df_merged.app_id=='com.ihappydate')  & (df_merged.install_date_utc>="2022-01-24")].copy()

#### Модель

In [ ]:
model_em = lgb.Booster(model_file='model_adrev_em.txt')
model_sm = lgb.Booster(model_file='model_adrev_sm.txt')
model_ih = lgb.Booster(model_file='model_adrev_ih.txt')

In [ ]:
df_em['predict_origin'] = model_em.predict(df_em[features+cat_features])
df_em['predict'] = df_em.apply(lambda x: max(x['predict_origin'], 0), axis=1)
df_em['count'] = df_em.apply(lambda x: 1, axis=1)
df_em['predict'] = df_em['predict'] + df_em['ad_revenue_0_13']

In [ ]:
df_sm['predict_origin'] = model_sm.predict(df_sm[features+cat_features])
df_sm['predict'] = df_sm.apply(lambda x: max(x['predict_origin'], 0), axis=1)
df_sm['count'] = df_sm.apply(lambda x: 1, axis=1)
df_sm['predict'] = df_sm['predict'] + df_sm['ad_revenue_0_13']

In [ ]:
df_ih['predict_origin'] = model_ih.predict(df_ih[features+cat_features])
df_ih['predict'] = df_ih.apply(lambda x: max(x['predict_origin'], 0), axis=1)
df_ih['count'] = df_ih.apply(lambda x: 1, axis=1)
df_ih['predict'] = df_ih['predict'] + df_ih['ad_revenue_0_13']